# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/flaviolin/DBTL1/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/flaviolin/DBTL1/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [20]:
import sys
sys.path.append('../')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [21]:
CYCLE = 3

user_params = {
    'media_file': f'../data/flaviolin/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../data/flaviolin/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../data/flaviolin/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [22]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
Well,,,,,,,,,,,,,,,
A1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014
B1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014
C1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014
D1,40.0,4.0,0.038344,20.0,0.196104,2.505369,0.06513,11.989143,3.103421,431.048016,0.000195,0.000292,0.000689,0.002345,0.000014
E1,40.0,4.0,0.038344,20.0,0.196104,2.505369,0.06513,11.989143,3.103421,431.048016,0.000195,0.000292,0.000689,0.002345,0.000014


## Create Line Description

In [23]:
df['Line Description'] = df.apply(create_media_description, axis=1)


## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [29]:
column_order = True if df.index[1][0] == 'B' else False

In [6]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'                                                                                              

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]

if column_order:
    df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
else:
    df['Line Name'] = df.apply(linefunction_row_order, axis=1)

In [7]:
df.head(2)

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",1,C3_WA1_C1-R1
B1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",2,C3_WA1_C1-R2


## Process measurement files

Read measurements file:

In [8]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1848,0.1326,0.1276,0.1281,0.1212,0.1489,0.1190,0.1478,0.0356,0.0463,0.0462,0.0459
B,0.1372,0.1491,0.1317,0.1302,0.1246,0.1542,0.1307,0.1408,0.0466,0.0467,0.0463,0.0457
C,0.1655,0.1797,0.1422,0.1535,0.1263,0.1738,0.1497,0.1529,0.0465,0.0466,0.0463,0.0456
D,0.1404,0.1507,0.1419,0.1342,0.1343,0.1365,0.1240,0.1378,0.0465,0.0467,0.0463,0.0458
E,0.1485,0.1421,0.1236,0.1162,0.1286,0.1339,0.1240,0.1156,0.0465,0.0463,0.0461,0.0456
F,0.1462,0.1491,0.1449,0.1108,0.1242,0.1323,0.1243,0.1144,0.0467,0.0461,0.0460,0.0458
G,0.0461,0.0464,0.0471,0.0466,0.0466,0.0462,0.0459,0.0466,0.0467,0.0462,0.0460,0.0457
H,0.0461,0.0465,0.0461,0.0458,0.0457,0.0479,0.0456,0.0458,0.0460,0.0462,0.0458,0.0455


In [9]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.8426,0.7256,0.6946,0.6547,0.6645,0.6620,0.7022,0.7407,0.1220,0.1351,0.1321,0.1316
B,0.7447,0.6553,0.7036,0.7253,0.7186,0.6918,0.7261,0.7712,0.1383,0.1368,0.1351,0.1304
C,0.7687,0.6767,0.7068,0.7064,0.6830,0.7775,0.7058,0.7952,0.1384,0.1376,0.1352,0.1311
D,0.7523,0.6914,0.6694,0.7097,0.6960,0.6981,0.6577,0.4033,0.1384,0.1373,0.1357,0.1318
E,0.7013,0.7183,0.7379,0.6883,0.7245,0.7083,0.6635,0.3975,0.1382,0.1374,0.1354,0.1319
F,0.7488,0.6886,0.7140,0.6862,0.7268,0.6922,0.6514,0.4125,0.1388,0.1362,0.1356,0.1317
G,0.1326,0.1347,0.1374,0.1374,0.1383,0.1370,0.1372,0.1381,0.1380,0.1363,0.1335,0.1309
H,0.1314,0.1334,0.1343,0.1342,0.1343,0.1340,0.1339,0.1339,0.1340,0.1346,0.1328,0.1297


### Process OD600

Normalize the data to the control well A9 value (water content):

In [10]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.1492,0.0970,0.0920,0.0925,0.0856,0.1133,0.0834,0.1122,0.0000,0.0107,0.0106,0.0103
B,0.1016,0.1135,0.0961,0.0946,0.0890,0.1186,0.0951,0.1052,0.0110,0.0111,0.0107,0.0101
C,0.1299,0.1441,0.1066,0.1179,0.0907,0.1382,0.1141,0.1173,0.0109,0.0110,0.0107,0.0100
D,0.1048,0.1151,0.1063,0.0986,0.0987,0.1009,0.0884,0.1022,0.0109,0.0111,0.0107,0.0102
E,0.1129,0.1065,0.0880,0.0806,0.0930,0.0983,0.0884,0.0800,0.0109,0.0107,0.0105,0.0100
F,0.1106,0.1135,0.1093,0.0752,0.0886,0.0967,0.0887,0.0788,0.0111,0.0105,0.0104,0.0102
G,0.0105,0.0108,0.0115,0.0110,0.0110,0.0106,0.0103,0.0110,0.0111,0.0106,0.0104,0.0101
H,0.0105,0.0109,0.0105,0.0102,0.0101,0.0123,0.0100,0.0102,0.0104,0.0106,0.0102,0.0099


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [11]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,1.492,0.970,0.920,0.925,0.856,1.133,0.834,1.122
B,1.016,1.135,0.961,0.946,0.890,1.186,0.951,1.052
C,1.299,1.441,1.066,1.179,0.907,1.382,1.141,1.173
D,1.048,1.151,1.063,0.986,0.987,1.009,0.884,1.022
E,1.129,1.065,0.880,0.806,0.930,0.983,0.884,0.800
F,1.106,1.135,1.093,0.752,0.886,0.967,0.887,0.788


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [12]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.7206,0.6036,0.5726,0.5327,0.5425,0.5400,0.5802,0.6187
B,0.6227,0.5333,0.5816,0.6033,0.5966,0.5698,0.6041,0.6492
C,0.6467,0.5547,0.5848,0.5844,0.5610,0.6555,0.5838,0.6732
D,0.6303,0.5694,0.5474,0.5877,0.5740,0.5761,0.5357,0.2813
E,0.5793,0.5963,0.6159,0.5663,0.6025,0.5863,0.5415,0.2755
F,0.6268,0.5666,0.5920,0.5642,0.6048,0.5702,0.5294,0.2905


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [13]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

df_600.index = df_600['Well']
df_340.index = df_340['Well']

df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

df_600.head()

,OD600
Well,
A1,1.492
B1,1.016
C1,1.299
D1,1.048
E1,1.129


Add measurements to the main dataframe:

In [14]:
measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Line Description,Replicate,Line Name,OD600,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",1,C3_WA1_C1-R1,1.492,0.7206
B1,40.0,4.0,0.028304,20.0,0.203775,2.142815,0.04985,10.682077,1.327616,400.020175,0.000282,0.001039,0.000786,0.002639,0.000014,"MOPS: 40.000000, Tricine: 4.000000, H3BO3: 0.0...",2,C3_WA1_C1-R2,1.016,0.6227


## Create EDD Experiment Description File

In [15]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [16]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [17]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [18]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
df['Measurement Type'] = 'Optical Density'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD600]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [19]:
measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)